![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [96]:
### Libraries ###
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

### Load an clean data ##
df = pd.read_csv('rental_info.csv')
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in x else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in x else 0)
#years = pd.get_dummies(df['release_year'], prefix='released')
#df = pd.concat([df, years], axis=1)
#X = df.drop(['release_year', 'rental_length_days', 'rental_date', 'return_date', 'special_features', 'length_2', 'amount_2', 'rental_rate_2'], axis=1)
X = df.drop(['rental_length_days', 'rental_date', 'return_date', 'special_features'], axis=1)
#, 'length_2', 'amount_2', 'rental_rate_2'
y = df['rental_length_days']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

### Standardize features ###
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Lasso Regression ###
lasso = Lasso(random_state=9)
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train_scaled, y_train)
best_lasso = lasso_grid.best_estimator_
lasso_pred = best_lasso.predict(X_test_scaled)
lasso_mse = mean_squared_error(y_test, lasso_pred)

### XGBoost Regression ###
xgb = XGBRegressor(random_state=9, objective='reg:squarederror')
xgb_params = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}
xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='neg_mean_squared_error')
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_
xgb_pred = best_xgb.predict(X_test)
xgb_mse = mean_squared_error(y_test, xgb_pred)

### Best Model ###
print(f"Lasso Regression MSE: {lasso_mse:.4f} (Best alpha: {best_lasso.alpha})")
print(f"XGBoost MSE: {xgb_mse:.4f} (Best params: {xgb_grid.best_params_})")

if xgb_mse < lasso_mse:
    best_model = best_xgb
    best_mse = xgb_mse
    print("\n XGBoost")
else:
    best_model = best_lasso
    best_mse = lasso_mse
    print("\n Lasso")

Lasso Regression MSE: 2.9417 (Best alpha: 0.001)
XGBoost MSE: 2.0651 (Best params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100})

 XGBoost
